# **Create The Byte-Pair Encoding (BPE) Tokenizer From Scratch**

As of 2025, BPE is still popular and is widely used. Models including GPT-2, GPT-3, GPT-4, Llama-3 etc. have made use fo this tokenizer. 

OpenAI's original implementation of the BPE tokenizer can be found [here](https://github.com/openai/gpt-2/blob/master/src/encoder.py), while practitioners usually incorporate the [tiktoken](https://github.com/openai/tiktoken) library in their model development pipelines. Karpathy's [minBPE](https://github.com/karpathy/minbpe) is also mentioned in Sebastian's work, as a possible alternative to the worflow below.

For practice, the BPE tokenizer will be implemented from scratch in this notebook - though it won't be nearly as optimized as OpenAI's or maybe even Karpathy's versions.



## **BPE Algorithm Outline**

>**1. Identify frequent pairs**
>- In each iteration, scan the text to find the most commonly occurring pair of bytes (or characters)
>
>**2. Replace and record**
>
>- Replace that pair with a new placeholder ID (one not already in use, e.g., if we start with 0...255, the first placeholder would be 256)
>- Record this mapping in a lookup table
>- The size of the lookup table is a hyperparameter, also called "vocabulary size" (for GPT-2, that's
>50,257)
>
>**3. Repeat until no gains**
>
>- Keep repeating steps 1 and 2, continually merging the most frequent pairs
>- Stop when no further compression is possible (e.g., no pair occurs more than once)
>
>**Decompression (decoding)**
>
>- To restore the original text, reverse the process by substituting each ID with its corresponding pair, using the lookup table
>

### **Working Example**

>&nbsp;
> Suppose we have the text (training dataset) `the cat in the hat` from which we want to build the vocabulary for a BPE tokenizer
>
>**Iteration 1**
>
>1. Identify frequent pairs
>  - In this text, "th" appears twice (at the beginning and before the second "e")
>
>2. Replace and record
>  - replace "th" with a new token ID that is not already in use, e.g., 256
>  - the new text is: `<256>e cat in <256>e hat`
>  - the new vocabulary is
>
>```
>  0: ...
>  ...
>  256: "th"
>```
>
>**Iteration 2**
>
>1. **Identify frequent pairs**  
>   - In the text `<256>e cat in <256>e hat`, the pair `<256>e` appears twice
>
>2. **Replace and record**  
>   - replace `<256>e` with a new token ID that is not already in use, for example, `257`.  
>   - The new text is:
>     ```
>     <257> cat in <257> hat
>     ```
>   - The updated vocabulary is:
>     ```
>     0: ...
>     ...
>     256: "th"
>     257: "<256>e"
>     ```
>
>**Iteration 3**
>
>1. **Identify frequent pairs**  
>   - In the text `<257> cat in <257> hat`, the pair `<257> ` appears twice (once at the beginning and once before “hat”).
>
>2. **Replace and record**  
>   - replace `<257> ` with a new token ID that is not already in use, for example, `258`.  
>   - the new text is:
>     ```
>     <258>cat in <258>hat
>     ```
>   - The updated vocabulary is:
>     ```
>     0: ...
>     ...
>     256: "th"
>     257: "<256>e"
>     258: "<257> "
>     ```
>     
>- and so forth
>
>&nbsp;
>#### Decoding Steps:
>
>- To restore the original text, we reverse the process by substituting each token ID with its corresponding pair in the reverse order they were introduced
>- Start with the final compressed text: `<258>cat in <258>hat`
>-  Substitute `<258>` → `<257> `: `<257> cat in <257> hat`  
>- Substitute `<257>` → `<256>e`: `<256>e cat in <256>e hat`
>- Substitute `<256>` → "th": `the cat in the hat`

## **Simplified BPE Implementation**

This is a simplified implementation of the BPE algorithm, which will mimic the `tiktoken` UI. Here the `encode()` method will approximate the original `train()` method.

In [3]:
from collections import Counter, deque
from functools import lru_cache
import json

In [ ]:
class BPETokenizerLocal:
    def __init__(self):
        # Map token_id to token_str   
        self.vocab = {}
        # Map token_str to token_od
        self.inverse_vocab = {}
        # Use a rank dict for GPT-2 merges. Low ranks have higher priority
        self.bpe_ranks = {}
     
    def train(self, text, vocab_size, allowed_special={"<|endoftext|>"}):
        """
        Train BPE tokenizer from scratch

        Args:
            text (str): Input / training text
            vocab_size (int): Desired vocabulary size
            allowed_special (set): Set of special tokens to include.
        """
        
        # Preprocessing: Replace spaces with "Ġ", as implemented in GPT-2.
        processed_text = []
        for i, char in enumerate(text):
            if char == " " and i != 0:
                processed_text.append("Ġ")
            if char != " ":
                processed_text.append(char)
        processed_text = "".join(processed_text)
        
        # Initialize vocab with unique characters, including "Ġ" if present starting
        # with first 256 ASCII characters
        unique_chars = [chr(i) for i in range(256)]
        unique_chars.extend(
            char for char in sorted(set(processed_text))
            if char not in unique_chars
        )
        if "Ġ" not in unique_chars:
            unique_chars.append("Ġ")
            
        self.vocab = {i: char for i, char in enumerate(unique_chars)}
        self.inverse_vocab = {char: i for i, char in self.vocab.items()}
        
        # Add allowed special tokens
        if allowed_special:
            for token in allowed_special:
                if token not in self.inverse_vocab:
                    new_id = len(self.vocab)
                    self.vocab[new_id] = token
                    self.inverse_vocab[token] = new_id
        
        # Tokenize the processed_text into token Ids
        token_ids = [self.inverse_vocab[char] for char in processed_text]
        
        # BPE steps: Repeatedly find and replace frequent pairs
        for new_id in range(len(self.vocab), vocab_size):
            pair_id = self.find_freq_pair(token_ids, mode='most')
            if pair_id is None:
                break
            token_ids = self.replace_pair(token_ids, pair_id, new_id)
            self.bpe_merges[pair_id] = new_id
            
        # Build vocab with merged tokens
        for (p0, p1), new_id in self.bpe_merges.items():
            merged_token = self.vocab[p0] + self.vocab[p1]
            self.vocab[new_id] = merged_token
            self.inverse_vocab[merged_token] = new_id
            
    def load_vocab_and_merges_from_openai(self, vocab_path, bpe_merges_path):
        """
        Load pretained vocabulary and BPE merges from OpenAI's GPT-2 files

        Args:
            vocab_path (str): Path to the vocab file (GPT-2 calls it 'encoder.json)
            bpe_merges_path (str): Path to bpe_merges file (GPT-2 calls it 'vocab.bpe'). 
        """
        # Load vocab
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            # Load vocab to correct format
            self.vocab = {int(v): k for k, v in loaded_vocab.items()}
            self.inverse_vocab = {k: int(v) for k, v in loaded_vocab.items()}
        
        # Handle newline character without adding a new token
        if "\n" not in self.inverse_vocab:
            # Use existing token ID as a placeholder for '\n' i.e. "<|endoftext|>" if available
            fallback_token = next((token for token in ["<|endoftext|>", "Ġ", ""] if token in self.inverse_vocab), None)
            if fallback_token is not None:
                newline_token_id = self.inverse_vocab[fallback_token]
            else:
                raise KeyError("No suitable token found in vocabulary to map '\\n'.")
            
            self.inverse_vocab["\n"] = newline_token_id
            self.vocab[newline_token_id]= "\n"
            
        # Load GPT-2 merges and store these with an assigned rank.
        self.bpe_ranks = {}
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            lines = file.readlines()
            if lines and lines[0].startswith("#"):
                lines = lines[1:]
            
            rank = 0
            for line in lines:
                pair = tuple(line.strip().split())
                if len(pair) == 2:
                    token1, token2 = pair
                    # if both tokens are not in vocab then skip
                    if token1 in self.inverse_vocab and token2 in self.inverse_vocab:
                        self.bpe_ranks[(token1, token2)] = rank
                        rank += 1
                    else:
                        print(f"Skipping pair {pair} since one token isn't in the vocabulary!")
                        
            
        

Signature: lru_cache(maxsize=128, typed=False)
Source:   
def lru_cache(maxsize=128, typed=False):
    """Least-recently-used cache decorator.

    If *maxsize* is set to None, the LRU features are disabled and the cache
    can grow without bound.

    If *typed* is True, arguments of different types will be cached separately.
    For example, f(3.0) and f(3) will be treated as distinct calls with
    distinct results.

    Arguments to the cached function must be hashable.

    View the cache statistics named tuple (hits, misses, maxsize, currsize)
    with f.cache_info().  Clear the cache and statistics with f.cache_clear().
    Access the underlying function with f.__wrapped__.

    See:  https://en.wikipedia.org/wiki/Cache_replacement_policies#Least_recently_used_(LRU)

    """

    # Users should only access the lru_cache through its public API:
    #       cache_info, cache_clear, and f.__wrapped__
    # The internals of the lru_cache are encapsulated for thread safety and
    #